## Retrieval

- Retrieval은 대량의 문서나 데이터에서 사용자의 질문이나 쿼리와 관련된 정보를 찾아내는 과정. 
- RAG (Retrieval-Augmented Generation) 시스템의 핵심 구성 요소.

#### 1. Document Loader (문서 로딩)
- 다양한 소스에서 문서를 불러오는 단계 
    - WebBaseLoader: 웹페이지 크롤링
    - PyPDFLoader: PDF 파일 읽기
    - 기타: CSV, JSON, 데이터베이스 등 다양한 형식 지원

#### 2. Embedding Model (임베딩)
- 텍스트를 벡터(숫자 배열)로 변환하는 단계 
    - 문서와 쿼리를 같은 벡터 공간에 매핑
    - 의미적으로 유사한 텍스트는 벡터 공간에서 가까운 위치에 배치
    - 예시: OpenAI의 text-embedding-3-small 모델 사용

#### 3. Vector Store (벡터 저장소)
- 벡터화된 문서를 저장하고 검색하는 단계 
    - FAISS, Chroma, Pinecone 등의 벡터 데이터베이스 사용
    - 빠른 유사도 검색 지원 (k-NN 알고리즘)
    - similarity_search(): 쿼리와 가장 유사한 문서 찾기

#### 작동 원리
1. 인덱싱 단계 (사전 작업):
    - 문서 로드 → 임베딩 → 벡터 스토어에 저장

2. 검색 단계 (런타임):
    - 사용자 쿼리 → 임베딩 → 벡터 스토어에서 유사 문서 검색
    - 코사인 유사도, 유클리드 거리 등으로 유사도 측정

#### 활용 사례
- Q&A 시스템: 관련 문서를 찾아 LLM에 컨텍스트로 제공
- 문서 검색: 대량의 문서에서 관련 정보 빠르게 탐색
- 추천 시스템: 유사한 콘텐츠 추천
- RAG: LLM의 답변에 실시간 정보 제공

In [1]:
from dotenv import load_dotenv
load_dotenv()  # .env 파일에서 환경 변수 로드

True

---
## 1. Document Loader

#### URL 기반

In [1]:
# WebBaseLoader : 웹페이지 내용을 읽어옴 (크롤링)
from langchain_community.document_loaders import WebBaseLoader

url = "https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%A0%95%EC%B1%85%EA%B3%BC_%EC%A7%80%EC%B9%A8"
loader = WebBaseLoader(url)

documents = loader.load()
documents[0].metadata['title']
print(documents[0].page_content[:1000])  # 문서 내용의 앞부분 출력

USER_AGENT environment variable not set, consider setting it to identify your requests.






위키백과:정책과 지침 - 위키백과, 우리 모두의 백과사전


























본문으로 이동







주 메뉴





주 메뉴
사이드바로 이동
숨기기



		둘러보기
	


대문최근 바뀜요즘 화제임의의 문서로





		사용자 모임
	


사랑방사용자 모임관리 요청





		편집 안내
	


소개도움말정책과 지침질문방



















검색











검색






















보이기
















기부

계정 만들기

로그인








개인 도구





기부 계정 만들기 로그인




























목차
사이드바로 이동
숨기기




처음 위치





1
최상위 정책








2
'정책과 지침'이란?








3
준수








4
집행








5
문서 내용








6
정책과 지침은 백과사전의 일부가 아닙니다








7
채택 과정




채택 과정 하위섹션 토글하기





7.1
제안과 채택








7.2
내용 변경






7.2.1
실질적인 변경










7.3
격하










8
같이 보기








9
외부 링크


















목차 토글







위키백과:정책과 지침



114개 언어




Afrikaansአማርኛअंगिकाالعربيةالدارجةمصرىঅসমীয়াАварAzərbaycancaتۆرکجهБашҡортсаBoarischБеларускаяБеларуская (тарашкевіца)БългарскиभोजपुरीBanjarবাংলাBrezhonegBosanskiCatalàНохчийнکوردیČeštinaCymraegDanskDeutschΕλληνικάEnglishEsperantoEspañolEuskaraفارسیSuomiFrançaisNordfriiskGaeilge贛語GalegoگیلکیBahasa Hulonta

#### PDF 기반

In [4]:
!pip install pypdf

  Using cached pypdf-6.1.3-py3-none-any.whl.metadata (7.1 kB)
Using cached pypdf-6.1.3-py3-none-any.whl (323 kB)



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# PyPDFLoader : PDF 파일 내용을 읽어옴
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("./data/The_Adventures_of_Tom_Sawyer.pdf")
documents = loader.load()
# print(documents[0].metadata)  # 문서 메타데이터 출력
print(documents[2].page_content[:1000])  # 문서 내용의 앞부분 출력

The Adventures of                 
Tom Sawyer 
 
MARK TWAIN 
Level 1 
 
Retold by Jacqueline Kehl                                                    
Series Editors: Andy Hopkins and Jocelyn Potter


---
### 2. Embedding Model

In [7]:
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")


In [8]:
######### 단일 문장 벡터화 테스트 #########

text = 'The quick brown fox jumps over the lazy dog.' 
vector = embedding_model.embed_query(text)

print(len(vector)) # 벡터의 차원 수 출력
print(vector)       # 벡터 출력


1536
[-0.01842353865504265, -0.00725775770843029, 0.0036669441033154726, -0.0542047917842865, -0.022724902257323265, 0.03694858402013779, 0.02903103083372116, 0.023866858333349228, 0.011229223571717739, -0.020618630573153496, -0.013360872864723206, 0.00907219760119915, -0.022636083886027336, 0.03540060296654701, -0.016177695244550705, 0.009116606786847115, -0.0117494473233819, -0.0562349334359169, -0.030604392290115356, -0.021722519770264626, -0.01834740862250328, -0.000452023814432323, 0.0019365651533007622, -0.0014123761793598533, -0.0006498833536170423, -0.00877401977777481, 0.006477422080934048, -0.03131494298577309, 0.008501219563186169, -0.0021601980552077293, 0.03344659134745598, -0.02941168285906315, -0.011368795298039913, -0.06460927426815033, -0.011007176712155342, -0.032507650554180145, -0.039841536432504654, -0.012605912983417511, -0.022103171795606613, -0.009034132584929466, 0.0008084882283583283, -0.061614811420440674, 0.03456316888332367, -0.05339273437857628, 0.06339118

In [9]:
######### 여러 문서를 벡터로 변환 #########

docs = [document.page_content for document in documents] 
vectors = embedding_model.embed_documents(docs)

print(len(vectors)) # 벡터의 개수 출력 (문서의 개수와 동일해야 함)
print(vectors[0])  # 첫 번째 문서의 벡터 출력


35
[0.01536838710308075, -0.03481070324778557, -0.009328825399279594, 0.014480682089924812, 0.007343378383666277, 0.014409349299967289, -0.05224776268005371, 0.049235906451940536, -0.01359297800809145, 0.015106831677258015, 0.008250897750258446, 0.03281337022781372, -0.041722118854522705, -0.015447646379470825, 0.020306244492530823, 0.06340747326612473, -0.045526567846536636, 0.027027437463402748, -0.0077634528279304504, 0.01865765079855919, 0.07437697052955627, 0.014821497723460197, -0.016676167026162148, -0.0307367742061615, 0.04010522738099098, -0.014940386638045311, 0.003134707221761346, -0.011730383150279522, 0.027534697204828262, -0.05890554562211037, 0.044797383248806, -0.04248300939798355, 0.00369546702131629, -0.018150391057133675, -0.018404019996523857, 0.056464359164237976, 0.024728916585445404, 0.00309309596195817, 0.025759287178516388, 0.026868918910622597, -0.0035904482938349247, 0.028866253793239594, -0.02845410443842411, 0.0370616689324379, 0.024237507954239845, 0.02017

### 3. Vector Store

In [ ]:
# !pip install faiss-cpu

In [10]:
from langchain_community.vectorstores import FAISS

# 벡터로 변환된 문서들을 FAISS 벡터 스토어에 저장
# - docs:  문서 내용 리스트(원본 텍스트)
# - vectors: 문서 벡터 리스트(임베딩 결과)
text_embeddings = list(zip(docs, vectors)) # (문서 내용, 벡터) 튜플 리스트 생성
vector_store = FAISS.from_embeddings(text_embeddings, embedding_model) # FAISS 벡터 스토어 생성
 
print(vector_store.index.ntotal)  # 벡터 스토어에 저장된 벡터의 총 개수 출력
 

35


In [11]:
# vector_store를 로컬에 저장하는 방법

vector_store.save_local("./db/faiss_vector_store")

# 저장된 벡터 스토어를 로드하는 방법
loaded_vector_store = FAISS.load_local(
    "./db/faiss_vector_store",  # 저장된 경로
    embedding_model, # 임베딩 모델
    allow_dangerous_deserialization=True # FAISS 벡터 스토어 로드 시 필요(보안 경고 무시)
    )

In [12]:
# similarity_search: 특정 쿼리와 유사한 문서 검색, k는 반환할 문서의 개수
result = vector_store.similarity_search("Tom Sawyer", k=3)  # "Tom Sawyer"와 유사한 상위 3개 문서 검색

result

[Document(id='d7167e23-39c8-43b8-a2b5-cb142422fefd', metadata={}, page_content='Introduction \n \n \nOne Saturday afternoon Tom wanted to have an adventure                    \nbecause he didn’t want to think about Injun Joe. He went \nto Huck and said, “I’m going to  look for treasure. Do you \nwant to come with me?” \n \nTom Sawyer loves adventures. He has a lot of adventures \nat home, at school, and with his friends. He has one \nadventure in a cave. But why is he there? What does he \nsee in the cave? And why is he afraid? \n \nMark Twain (1835-1910) is a famous American writer. \nHis name was Samuel Clemens. Young Samuel lived in \nHannibal, Missouri, a small town on the Mississippi River. \nHe loved the river and he liked watching the big boats                  \non it. \nSamuel loved adventures. He worked on boats on the \nMississippi River for two years. Then he went to Nevada. \nHe looked for treasure, but he didn’t find it. He worked for \na newspaper there. His stories were